In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").getOrCreate()

In [17]:
from pyspark.sql.functions import *

In [3]:
# NumPy ve Pandas kütüphanelerini içe aktarma
import numpy as np
import pandas as pd

# Rastgele sayılar içeren bir NumPy dizisi oluşturma
data = np.random.randint(1, 100, size=(100, 4))

# Pandas DataFrame'i oluşturmak için NumPy dizisini kullanma
df_pandas = pd.DataFrame(data, columns=["col1", "col2", "col3", "col4"])

# Pandas DataFrame'i Spark DataFrame'e dönüştürme
df_spark = spark.createDataFrame(df_pandas)


# Veri kümesini görüntülemek için print işlemini kullanma
df_spark.show()


+----+----+----+----+
|col1|col2|col3|col4|
+----+----+----+----+
|  50|  11|  29|  75|
|  67|   9|  80|  92|
|  15|  23|  83|  54|
|  74|  25|  74|  50|
|   9|  17|  75|  40|
|  84|  91|  52|  50|
|  90|   5|  38|  98|
|  89|  29|  57|  98|
|   3|  68|  77|  48|
|  84|  54|  14|  59|
|  90|  26|  85|  55|
|  42|   4|  87|  10|
|  46|  75|  50|  79|
|  31|  78|  47|  27|
|  44|  72|  92|  81|
|  52|  82|  64|  87|
|  69|  60|  71|   7|
|  75|  34|  70|  61|
|  32|  94|  49|  21|
|  29|  65|  96|  82|
+----+----+----+----+
only showing top 20 rows



In [4]:
df_spark.write.json("hdfs://namenode:9000/training_samples/first_dataframe/1.json")

In [5]:
df_spark.write.parquet("hdfs://namenode:9000/training_samples/first_dataframe_parquet/")

In [18]:
df_haber = spark.read.format("csv").option("header","true").load("exceptions.csv")
df_haber.select(col("MAIN_SOURCE")).distinct().show()

+---------------+
|    MAIN_SOURCE|
+---------------+
|           SEAT|
|VOLKSWAGEN A.G.|
|   SCANIA CV AB|
+---------------+



In [20]:
df_haber.write.parquet("hdfs://namenode:9000/training_samples/second_dataframe_parquet/")

In [21]:
df_haber.write \
.option("compression","gzip") \
.partitionBy(["MAIN_SOURCE","ORDER_TYPE"]) \
.parquet("hdfs://namenode:9000/training_samples/second_dataframe_parquet_with_partition/")

In [27]:
df_exceptions = spark.read.parquet("hdfs://namenode:9000/training_samples/second_dataframe_parquet_with_partition")
df_exceptions.printSchema()

df_exceptions.where(col("MAIN_SOURCE")=="SEAT").show()

root
 |-- _c0: string (nullable = true)
 |-- ORDER_NO: string (nullable = true)
 |-- PRODUCT_CODE: string (nullable = true)
 |-- ORDER_TYPE: string (nullable = true)
 |-- TRANSFER_TYPE: string (nullable = true)
 |-- TRANSACTION_ID: string (nullable = true)
 |-- TRANSACTION_TYPE: string (nullable = true)
 |-- TRANSACTION_DATE: string (nullable = true)
 |-- PENDING_TRANSACTION_TYPE: string (nullable = true)
 |-- average_days: string (nullable = true)
 |-- NEXT_TRXN_DATE_FORECAST: string (nullable = true)
 |-- MAIN_SOURCE: string (nullable = true)

+---+-------------+--------------+----------+-------------+--------------------+--------------------+----------------+------------------------+------------+-----------------------+-----------+
|_c0|     ORDER_NO|  PRODUCT_CODE|ORDER_TYPE|TRANSFER_TYPE|      TRANSACTION_ID|    TRANSACTION_TYPE|TRANSACTION_DATE|PENDING_TRANSACTION_TYPE|average_days|NEXT_TRXN_DATE_FORECAST|MAIN_SOURCE|
+---+-------------+--------------+----------+-------------+---

In [28]:
df_haber.write \
.option("compression","gzip") \
.partitionBy(["MAIN_SOURCE","ORDER_TYPE"]) \
.parquet("hdfs://namenode:9000/training_samples/second_dataframe_parquet_with_partition2/")

In [29]:
df_haber.write.csv("\deneme\\abc.csv")

In [32]:
df_total = df_exceptions.groupBy("ORDER_TYPE").agg(count(col("ORDER_NO")).alias("count"))

df_total.write.json("hdfs://namenode:9000/training_samples/third_dataframe_json/file.json")